In [40]:
# --- Cell 1: Env, paths, .env loader, tokens (robust) ---
import os, json, re, time
from pathlib import Path
from datetime import date, timedelta
import pandas as pd

# Optional dotenv
try:
    from dotenv import load_dotenv, find_dotenv
except Exception:
    load_dotenv = None
    find_dotenv = None

def mask(s: str | None) -> str:
    if not s: return "<missing>"
    return (s[:4] + "…" + s[-4:]) if len(s) > 8 else "***"

# Resolve repo root (works from / or /scripts)
cwd = Path.cwd().resolve()
repo_root = next((p for p in [cwd, *cwd.parents] if (p / ".git").exists() or p.name == "spending-dashboard"), cwd)

# ✅ CI-safe override: prefer GitHub workspace path if present
gw = os.getenv("GITHUB_WORKSPACE")
if gw:
    repo_root = Path(gw).resolve()

# Load .env if present (scripts/.env preferred)
def load_envs():
    if load_dotenv is None:
        return
    abs_override = os.getenv("ENV_PATH", str(repo_root / "scripts" / ".env"))
    if abs_override and Path(abs_override).exists():
        try:
            load_dotenv(abs_override, override=False, encoding="utf-8")
        except TypeError:
            load_dotenv(abs_override, override=False)
    for p in [
        repo_root / "scripts" / ".env",
        repo_root / ".env",
        repo_root / "config" / ".env",
        cwd / ".env",
    ]:
        if Path(p).exists():
            try:
                load_dotenv(str(p), override=False, encoding="utf-8")
            except TypeError:
                load_dotenv(str(p), override=False)
    if find_dotenv:
        found = find_dotenv(usecwd=True)
        if found:
            try:
                load_dotenv(found, override=False, encoding="utf-8")
            except TypeError:
                load_dotenv(found, override=False)

load_envs()

# Normalize env
PLAID_CLIENT_ID = os.getenv("PLAID_CLIENT_ID")
PLAID_SECRET    = os.getenv("PLAID_SECRET")
PLAID_ENV       = (os.getenv("PLAID_ENV", "production") or "production").strip().lower()
alias = {"prod":"production","live":"production","dev":"development","devel":"development","sb":"sandbox"}
PLAID_ENV = alias.get(PLAID_ENV, PLAID_ENV)
if PLAID_ENV not in {"production","development","sandbox"}:
    PLAID_ENV = "production"

# Paths (env-overridable)
OUTPUT_DIR = Path(os.getenv("OUTPUT_DIR", str(repo_root / "data" / "raw")))
STATE_DIR  = Path(os.getenv("STATE_DIR",  str(repo_root / ".state")))
TOKENS_PATH = Path(os.getenv("TOKENS_PATH", str(STATE_DIR / "access_tokens.json")))
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
STATE_DIR.mkdir(parents=True, exist_ok=True)

# --- Load & validate access tokens (env > file), canonical-only ---
def _strip_bom(s: str) -> str:
    return s.lstrip("\ufeff") if isinstance(s, str) else s

def _parse_pairs_blob(blob: str) -> dict:
    raw = [p.strip() for sep in ["\n",";","|",","] for p in (blob.split(sep) if sep in blob else []) if p.strip()]
    if not raw: raw = [blob.strip()]
    out = {}
    for p in raw:
        if "=" in p:
            k, v = p.split("=", 1)
        elif ":" in p:
            k, v = p.split(":", 1)
        else:
            continue
        k = k.strip().strip('"').strip("'")
        v = v.strip().strip('"').strip("'")
        if k and v:
            out[k] = v
    return out

def _normalize_tokens(obj) -> dict:
    if isinstance(obj, dict):
        return {str(k): str(v).strip() for k,v in obj.items()}
    if isinstance(obj, list):
        out = {}
        for item in obj:
            if isinstance(item, dict):
                name = item.get("issuer") or item.get("bank") or item.get("name")
                token = item.get("access_token") or item.get("token")
                if name and token:
                    out[str(name)] = str(token).strip()
        return out
    if isinstance(obj, str):
        s = _strip_bom(obj).strip()
        try:
            parsed = json.loads(s)    # JSON first
            return _normalize_tokens(parsed)
        except Exception:
            return _parse_pairs_blob(s)
    return {}

def load_access_tokens():
    blob = os.getenv("PLAID_ACCESS_TOKENS", "").strip()
    if blob:
        tokens = _normalize_tokens(blob)
        if tokens:
            return tokens
    if TOKENS_PATH.exists():
        raw = TOKENS_PATH.read_text(encoding="utf-8", errors="ignore")
        tokens = _normalize_tokens(raw)
        if tokens:
            return tokens
    raise AssertionError(
        f"Could not load access tokens. Provide PLAID_ACCESS_TOKENS env or a valid JSON at {TOKENS_PATH}."
    )

ACCESS_TOKENS = load_access_tokens()

PAT = re.compile(r"^access-(?:production|development|sandbox)-[a-z0-9\-]+$")
expected_prefix = f"access-{PLAID_ENV}-"
bad = [k for k,v in ACCESS_TOKENS.items() if not isinstance(v, str) or not v.startswith(expected_prefix) or not PAT.match(v)]
assert not bad, f"Non-canonical or wrong-env tokens for: {bad}. Ensure tokens look like '{expected_prefix}…' (no '/', '+', '=')."

print(
    "Env OK →",
    "PLAID_CLIENT_ID:", mask(PLAID_CLIENT_ID),
    "| PLAID_SECRET:", mask(PLAID_SECRET),
    "| PLAID_ENV:", PLAID_ENV,
    "| OUTPUT_DIR:", str(OUTPUT_DIR),
    "| TOKENS_PATH:", str(TOKENS_PATH),
)
print(f"Loaded {len(ACCESS_TOKENS)} token(s).")

Env OK → PLAID_CLIENT_ID: 68bb…6689 | PLAID_SECRET: a605…7df5 | PLAID_ENV: production | OUTPUT_DIR: C:\Users\kosis\Downloads\Automation\spending-dashboard\data\raw | TOKENS_PATH: C:\Users\kosis\Downloads\Automation\spending-dashboard\.state\access_tokens.json
Loaded 3 token(s).


In [41]:
# --- Cell 2: Plaid client init (v10+ preferred, legacy fallback) ---
USE_PLAID_V10 = False
client = None

try:
    # v10+ path
    from plaid.api import plaid_api
    from plaid.configuration import Configuration
    try:
        from plaid.configuration import Environment  # newer enum
        env_host = {
            "production":  Environment.Production,
            "development": Environment.Development,
            "sandbox":     Environment.Sandbox,
        }[PLAID_ENV]
        config = Configuration(host=env_host)
    except Exception:
        # fallback if Environment enum not present
        host_url = {
            "production":  "https://production.plaid.com",
            "development": "https://development.plaid.com",
            "sandbox":     "https://sandbox.plaid.com",
        }[PLAID_ENV]
        config = Configuration(host=host_url)

    from plaid.api_client import ApiClient
    config.api_key["clientId"] = PLAID_CLIENT_ID
    config.api_key["secret"]   = PLAID_SECRET
    api_client = ApiClient(config)
    client = plaid_api.PlaidApi(api_client)
    USE_PLAID_V10 = True
    print("Plaid SDK: v10+ (plaid_api)")
except Exception as e_v10:
    try:
        # legacy path
        from plaid import Client as LegacyClient
        client = LegacyClient(
            client_id=PLAID_CLIENT_ID,
            secret=PLAID_SECRET,
            environment=PLAID_ENV
        )
        USE_PLAID_V10 = False
        print("Plaid SDK: legacy Client()")
    except Exception as e_legacy:
        raise ImportError(
            "Could not initialize Plaid client. Ensure 'plaid-python' is installed. "
            f"v10 error: {e_v10}\nlegacy error: {e_legacy}"
        )

# Optional quick probe (set PRECHECK=1 to enable)
if os.getenv("PRECHECK", "0") == "1" and USE_PLAID_V10:
    from plaid.model.accounts_get_request import AccountsGetRequest
    from plaid.api_client import ApiException
    for issuer, tok in ACCESS_TOKENS.items():
        try:
            n = len(client.accounts_get(AccountsGetRequest(access_token=tok)).to_dict().get("accounts", []))
            print(f"{issuer}: ✅ accounts_get OK ({n} accounts)")
        except ApiException as e:
            print(f"{issuer}: ❌ API {e.status} -> {getattr(e, 'body', e)}")

Plaid SDK: v10+ (plaid_api)


In [42]:
# --- Cell 3: Pull & consolidate via /transactions/sync (fallback to GET) ---
from pathlib import Path
import numpy as np
import pandas as pd

# Rolling window (change 90 → 180/365 if you want a bigger window)
DAYS_BACK = int(os.getenv("DAYS_BACK", "180"))
end_date = date.today()
start_date = end_date - timedelta(days=DAYS_BACK)

CURSORS_PATH = STATE_DIR / "plaid_cursors.json"

def load_cursors() -> dict:
    if CURSORS_PATH.exists():
        try:
            return json.loads(CURSORS_PATH.read_text(encoding="utf-8"))
        except Exception:
            pass
    return {}

def save_cursors(cur: dict):
    CURSORS_PATH.parent.mkdir(parents=True, exist_ok=True)
    CURSORS_PATH.write_text(json.dumps(cur, ensure_ascii=False, indent=2), encoding="utf-8")

def normalize_category(x):
    return " > ".join(x) if isinstance(x, (list, tuple)) else x

# Convert raw txn dicts to our normalized schema
def df_from_txns(txns: list[dict], bank_name: str) -> pd.DataFrame:
    if not txns:
        return pd.DataFrame()
    df = pd.DataFrame(txns)

    # Ensure expected columns exist
    expected_cols = [
        "name","merchant_name","payment_channel","pending",
        "account_id","transaction_id","category","date","amount"
    ]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = None

    # Normalize
    df["category"] = df["category"].apply(normalize_category)
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")

    # Bank & card
    df["bank_name"] = bank_name
    df["card_name"] = bank_name  # upgrade later via accounts_dim

    keep_cols = [
        "date","name","merchant_name","category","amount",
        "payment_channel","pending","account_id","transaction_id",
        "bank_name","card_name"
    ]
    return df[[c for c in keep_cols if c in df.columns]].copy()

# Read previous latest.csv (acts as our on-repo cache)
latest_csv_path = (repo_root / "data" / "raw" / "latest.csv")
prev = pd.DataFrame()
if latest_csv_path.exists():
    try:
        prev = pd.read_csv(latest_csv_path)
        prev["date"] = pd.to_datetime(prev["date"], errors="coerce")
        # Ensure we always have the column even if an older file omitted it
        if "transaction_id" not in prev.columns:
            prev["transaction_id"] = pd.Series(dtype=object)
    except Exception as e:
        print(f"⚠️ Could not read previous latest.csv: {e}")

cursors = load_cursors()
all_added_mod = []   # list of DataFrames of added/modified across banks
all_removed_ids = set()

if 'USE_PLAID_V10' in globals() and USE_PLAID_V10:
    # --- Sync-first path using Plaid v10 client ---
    from plaid.model.transactions_sync_request import TransactionsSyncRequest

    def sync_one(bank_name: str, access_token: str, cursor: str | None):
        """Call /transactions/sync until has_more is False. Omit 'cursor' on first call."""
        added, modified, removed_ids = [], [], []
        next_cursor = cursor
        while True:
            req_kwargs = {"access_token": access_token, "count": 500}
            if isinstance(next_cursor, str) and next_cursor:
                req_kwargs["cursor"] = next_cursor
            req = TransactionsSyncRequest(**req_kwargs)

            resp = client.transactions_sync(req).to_dict()
            added.extend(resp.get("added", []) or [])
            modified.extend(resp.get("modified", []) or [])

            # removed may be list[str] or list[dict]
            rem = resp.get("removed", []) or []
            for r in rem:
                if isinstance(r, dict):
                    rid = r.get("transaction_id")
                    if rid: removed_ids.append(rid)
                elif isinstance(r, str):
                    removed_ids.append(r)

            next_cursor = resp.get("next_cursor", next_cursor)
            if not resp.get("has_more", False):
                break

        return (next_cursor if isinstance(next_cursor, str) and next_cursor else None,
                added, modified, removed_ids)

    for bank_name, token in ACCESS_TOKENS.items():
        print(f"🔄 SYNC {bank_name} (start: {'cursor-present' if cursors.get(token) else 'no-cursor'})")
        cur0 = cursors.get(token)
        next_cur, added, modified, removed_ids = sync_one(bank_name, token, cur0)

        df_add = df_from_txns(added, bank_name)
        df_mod = df_from_txns(modified, bank_name)
        all_added_mod.append(df_add)
        all_added_mod.append(df_mod)
        all_removed_ids.update(removed_ids)

        cursors[token] = next_cur
        print(f"   → added={len(df_add):,}, modified={len(df_mod):,}, removed={len(removed_ids):,}, next_cursor={'set' if next_cur else 'None'}")

    # Start from previous CSV and apply deltas
    cur = prev.copy()

    # Remove deleted transaction_ids (guard if column is present)
    if not cur.empty and all_removed_ids and "transaction_id" in cur.columns:
        cur = cur[~cur["transaction_id"].astype(str).isin({str(x) for x in all_removed_ids})]
    elif not cur.empty and all_removed_ids:
        print("ℹ️ Skipping delete-apply: previous cache lacks 'transaction_id' column.")

    # Replace modified ids and add new ones
    if any(len(x) for x in all_added_mod):
        new_mod = (pd.concat([df for df in all_added_mod if not df.empty], ignore_index=True)
                   if all_added_mod else pd.DataFrame())
        if not new_mod.empty and "transaction_id" not in new_mod.columns:
            new_mod["transaction_id"] = pd.Series(dtype=object)

        if not cur.empty and "transaction_id" in cur.columns and "transaction_id" in new_mod.columns:
            mod_ids = set(new_mod["transaction_id"].dropna().astype(str).tolist())
            if mod_ids:
                cur = cur[~cur["transaction_id"].astype(str).isin(mod_ids)]
        else:
            if not cur.empty and not new_mod.empty:
                print("ℹ️ Skipping modify-replace: one frame lacks 'transaction_id'; appending only.")

        cur = pd.concat([cur, new_mod], ignore_index=True) if not new_mod.empty else cur

    combined = cur.copy()

else:
    # --- Fallback: windowed GET per item (legacy client) ---
    def fetch_transactions_get(bank_name: str, access_token: str) -> pd.DataFrame:
        txns = []
        offset = 0
        while True:
            resp = client.Transactions.get(
                access_token=access_token,
                start_date=start_date,
                end_date=end_date,
                options={"count": 500, "offset": offset}
            )
            total = resp["total_transactions"]
            txns.extend(resp["transactions"])
            if len(txns) >= total:
                break
            offset = len(txns)
            if offset > 50_000:
                raise RuntimeError(f"Pagination runaway for {bank_name}")
        return df_from_txns(txns, bank_name)

    frames = []
    for bank_name, token in ACCESS_TOKENS.items():
        print(f"🔄 GET {bank_name} ({start_date} → {end_date})…")
        frames.append(fetch_transactions_get(bank_name, token))
    combined = pd.concat([f for f in frames if f is not None and not f.empty], ignore_index=True) if frames else pd.DataFrame()

# Finalize for both paths: trim to window, sort, dedupe by transaction_id if present
if combined is None or combined.empty:
    combined = pd.DataFrame(columns=[
        "date","name","merchant_name","category","amount",
        "payment_channel","pending","account_id","transaction_id",
        "bank_name","card_name"
    ])
else:
    combined["date"] = pd.to_datetime(combined["date"], errors="coerce")
    combined = combined[
        (combined["date"] >= pd.Timestamp(start_date)) &
        (combined["date"] <= pd.Timestamp(end_date))
    ]
    if "transaction_id" in combined.columns:
        combined = (combined
                    .sort_values("date", ascending=False)
                    .drop_duplicates(subset=["transaction_id"], keep="first")
                    .reset_index(drop=True))
    else:
        combined = combined.sort_values("date", ascending=False).reset_index(drop=True)

# Save cursors (persist locally; CI keeps it in the workspace unless you choose to cache/commit)
try:
    save_cursors(cursors)
except Exception as e:
    print(f"⚠️ Could not save cursors: {e}")

print(f"✅ Consolidated using {'SYNC' if 'USE_PLAID_V10' in globals() and USE_PLAID_V10 else 'GET'} → rows={len(combined):,} across {len(ACCESS_TOKENS)} bank(s)")
print(f"Window: {start_date} → {end_date} | DAYS_BACK={DAYS_BACK}")


🔄 SYNC Discover (start: cursor-present)
   → added=0, modified=0, removed=0, next_cursor=set
🔄 SYNC Petal (start: cursor-present)
   → added=0, modified=0, removed=0, next_cursor=set
🔄 SYNC Silver State Schools Credit Union (start: cursor-present)
   → added=0, modified=0, removed=0, next_cursor=set
✅ Consolidated using SYNC → rows=0 across 3 bank(s)
Window: 2025-03-16 → 2025-09-12 | DAYS_BACK=180


In [43]:
# --- Cell 4: Clean -> normalize schema ---
if combined.empty:
    # Create an empty but well-typed frame to keep Power BI stable
    combined = pd.DataFrame(columns=[
        "date","name","merchant_name","category","amount","payment_channel","pending",
        "account_id","transaction_id","bank_name","card_name"
    ])

# Normalize category: Plaid sometimes returns list; make it a short string
if "category" in combined.columns:
    combined["category"] = combined["category"].apply(
        lambda x: " > ".join(x) if isinstance(x, (list, tuple)) else x
    )

# Ensure date type
combined["date"] = pd.to_datetime(combined["date"], errors="coerce")

# Keep only expected columns (but don’t error if some are missing)
keep_cols = [
    "date","name","merchant_name","category","amount",
    "payment_channel","pending","account_id","transaction_id",
    "bank_name","card_name"
]
combined = combined[[c for c in keep_cols if c in combined.columns]].copy()

# Sort newest first
combined = combined.sort_values("date", ascending=False).reset_index(drop=True)

# Fill minimal NA for downstream friendliness
for c in ["name","merchant_name","category","payment_channel","bank_name","card_name"]:
    if c in combined.columns:
        combined[c] = combined[c].fillna("")


In [44]:
# --- Cell 5: YAML helpers (merchant key, mapping, non-spend) ---
from pathlib import Path
import re
import yaml

def merchant_key_from(name: str) -> str:
    s = (name or "").upper()
    s = re.sub(r"APPLE PAY ENDING IN \d{4}", "", s)
    s = re.sub(r"#\d{2,}", "", s)              # strip store numbers like #1234
    s = re.sub(r"\d+", "", s)                  # kill stray digits
    s = re.sub(r"[^A-Z&\s]", " ", s)           # keep letters, ampersand, spaces
    s = re.sub(r"\s+", " ", s).strip()
    return s

def apply_yaml_mapping(df: pd.DataFrame, ymap: dict) -> pd.DataFrame:
    if not ymap or df.empty:
        # Still add standard columns so schema is stable
        out = df.copy()
        for c in ["display_name_final","category_final","subcategory_final","tags_final","confidence_final","source_final"]:
            if c not in out.columns:
                out[c] = None
        out["source_final"] = out["source_final"].fillna("raw")
        out["confidence_final"] = out["confidence_final"].fillna("raw")
        return out

    look = {k.upper(): v for k, v in ymap.items()}
    rows = []
    # (kept for clarity; merge-based vectorization is overkill at this size)
    for _, r in df.iterrows():
        mk = r.get("merchant_key", "")
        m = look.get(mk, {})
        rows.append({
            **r,
            "display_name_final": m.get("display_name", r.get("merchant_name") or r.get("name")),
            "category_final":     m.get("category"),
            "subcategory_final":  m.get("subcategory"),
            "tags_final":         ",".join(m.get("tags", [])) if isinstance(m.get("tags", []), (list, tuple)) else m.get("tags"),
            "confidence_final":   m.get("confidence", "map"),
            "source_final":       "yaml" if m else "raw"
        })
    return pd.DataFrame(rows)

def mark_non_spend_flows(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty: return df
    pats = [
        r"PAYMENT", r"TRANSFER", r"ACH", r"ZELLE", r"DIRECTPAY", r"CREDIT",
        r"REFUND", r"REIMBURSE", r"ADJUSTMENT", r"REVERSAL"
    ]
    pat = re.compile("|".join(pats))
    names = (df.get("name", pd.Series("", index=df.index)).fillna("") + " " +
             df.get("merchant_name", pd.Series("", index=df.index)).fillna("")).str.upper()
    df = df.copy()
    df["is_non_spend_flow"] = names.str.contains(pat)
    return df


In [45]:
# --- Cell 6: Optional YAML enrichment, then finalize columns ---
# Build a robust merchant key
combined["merchant_key"] = combined["merchant_name"].where(
    combined["merchant_name"].astype(str).str.len() > 0,
    combined["name"]
).map(merchant_key_from)

# Load YAML map if exists (use repo_root)
PATH_YAML = (repo_root / "config" / "categories.yaml")
ymap = {}
if PATH_YAML.exists():
    with open(PATH_YAML, "r", encoding="utf-8") as f:
        ymap = yaml.safe_load(f) or {}

# Apply mapping + mark non-spend flows
enriched = apply_yaml_mapping(combined, ymap)
enriched = mark_non_spend_flows(enriched)

# ⚠️ FIXED: missing comma in your original list between card_name and display_name_final
cols = [
    "date","name","merchant_name","merchant_key","category","amount",
    "bank_name","card_name",
    "display_name_final","category_final","subcategory_final","tags_final",
    "is_non_spend_flow","confidence_final","source_final"
]
for c in cols:
    if c not in enriched.columns:
        enriched[c] = None
enriched = enriched[cols].copy()

# Keep dates as date (or datetime) for Power BI
enriched["date"] = pd.to_datetime(enriched["date"], errors="coerce")


In [46]:
# --- Cell 7: Write latest.csv + preview ---
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
latest_path = OUTPUT_DIR / "latest.csv"

# Write enriched directly (so Power BI gets the good stuff)
enriched.to_csv(latest_path, index=False)

# Sanity
assert latest_path.exists(), "latest.csv was not written."
assert "bank_name" in enriched.columns, "bank_name column missing."
assert "card_name" in enriched.columns, "card_name column missing."

print(f"✅ Latest CSV saved → {latest_path}  rows={len(enriched):,}")
try:
    print("\nPreview (top 10):")
    print(enriched.head(10).to_string(index=False))
except Exception:
    pass

✅ Latest CSV saved → C:\Users\kosis\Downloads\Automation\spending-dashboard\data\raw\latest.csv  rows=0

Preview (top 10):
Empty DataFrame
Columns: [date, name, merchant_name, merchant_key, category, amount, bank_name, card_name, display_name_final, category_final, subcategory_final, tags_final, is_non_spend_flow, confidence_final, source_final]
Index: []
